In [2]:
from PIL import Image
import os, glob

#이미지 224x224에서 -> 28x28로 변환하기
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/scissor"
resize_images(image_dir_path)

print("가위 이미지 resize 완료!")

# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/rock"
resize_images(image_dir_path)

print("바위 이미지 resize 완료!")


# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/paper"
resize_images(image_dir_path)

print("보 이미지 resize 완료!")


100  images to be resized.
100  images resized.
가위 이미지 resize 완료!
100  images to be resized.
100  images resized.
바위 이미지 resize 완료!
100  images to be resized.
100  images resized.
보 이미지 resize 완료!


In [11]:
#가위, 바위, 보 데이터를 읽을 수 있는 load_data() 함수 만들기
#load_data() 함수는 입력으로 이미지가 있는 폴더 위치를 받음
#가위바위보 -> 3개의 클래스 -> 가위 0, 바위 1, 보 2

import numpy as np

def load_data(img_path, number_of_data=300):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 300 입니다.
x_train shape: (300, 28, 28, 3)
y_train shape: (300,)


라벨:  0


In [12]:
import matplotlib.pyplot as plt
plt.imshow(x_train[0])
print('라벨: ', y_train[0])

In [17]:
#딥러닝 네트워크 설계하기

import tensorflow as tf
from tensorflow import keras
import numpy as np

# model을 직접 만들어 보세요.
# Hint! model의 입력/출력부에 특히 유의해 주세요. 

#가위바위보 데이터셋은 MNIST 데이터셋과 어떤 점이 달라졌나요?
# [[YOUR CODE]]

from tensorflow import keras

#channel : 정해진 값을 가진 건 아님, 
#1과2는 두배 간격이 좋음 (16,32가 정확도가 가장 높게 나옴. )
n_channel_1=16
n_channel_2=32
n_channel_3=64
n_dense=64
n_train_epoch=45

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())

#첫번째 dsnse레이어의 첫번째 인자 : 클래스 3개니까, 16으로 바꿔야 하나...?
#16,32,64,128 바이트?의 단위 같은데, 뭘 의미하는지 모르겠어
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

print('Model에 추가된 Layer 개수: ', len(model.layers))

model.summary()



Model에 추가된 Layer 개수:  7
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 26, 26, 16)        448       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                51264     
_________________________________________________________________
dense_7 (Dense)              (

In [18]:
#딥러닝 네트워크 학습시키기
# Hint! model.compile()과 model.fit()을 사용해 봅시다.


model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

#정규화된 x_train값을 넣어야해서 norm으로 -> 근데 그냥 x_train으로 하면 다 0,1이 나옴 왜그럴까??
model.fit(x_train_norm, y_train, epochs=n_train_epoch)

#과대적합 : 트레이닝 데이터, 테스트 데이터 > 학습시킨 데이터에 대한 모델로만 
#에포크를 너무 높였기 때문 (훈련데이터로만 계속 훈련을 하여서)
# <<- 정확도가 오르락 내리락하는게 과대 적합이라고 생각했는데, 아니였음 !!! 

#왜 정확도(accuracy)가 오르락 내리락 할까? : 경사 하강법(Gradient descent)을 통해서 loss를 줄이면서
#최적합 w값을 찾아가는 과정에 있어서 한번씩 값이 튀는 지점이 있기 때문 
#과녁의 이미지에서 에포크 한번 = 1번 학습하면서 과녁의 중앙=최적값으로 다가가는데, 그때 정확도 값이 오르락 내리락 하는 것 
#자세한 이해를 돕기위한 이미지 링크 : https://thebook.io/080228/part03/ch09/02/

#그럼 과적합 되었다는건 어떻게 아는걸까?

Epoch 1/45
10/10 [==============================] - 0s 3ms/step - loss: 1.1335 - accuracy: 0.2693
Epoch 2/45
10/10 [==============================] - 0s 3ms/step - loss: 1.0916 - accuracy: 0.3943
Epoch 3/45
10/10 [==============================] - 0s 3ms/step - loss: 1.0778 - accuracy: 0.4886
Epoch 4/45
10/10 [==============================] - 0s 3ms/step - loss: 1.0573 - accuracy: 0.4436
Epoch 5/45
10/10 [==============================] - 0s 3ms/step - loss: 1.0472 - accuracy: 0.5612
Epoch 6/45
10/10 [==============================] - 0s 3ms/step - loss: 1.0219 - accuracy: 0.6035
Epoch 7/45
10/10 [==============================] - 0s 3ms/step - loss: 0.9733 - accuracy: 0.6125
Epoch 8/45
10/10 [==============================] - 0s 3ms/step - loss: 0.9460 - accuracy: 0.6831
Epoch 9/45
10/10 [==============================] - 0s 3ms/step - loss: 0.8916 - accuracy: 0.6725
Epoch 10/45
10/10 [==============================] - 0s 3ms/step - loss: 0.8795 - accuracy: 0.6766
Epoch 11/45
10/10 [

In [19]:
#test
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test_set/scissor"
resize_images(image_dir_path)

print("가위 이미지 resize 완료!")

# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test_set/rock"
resize_images(image_dir_path)

print("바위 이미지 resize 완료!")


# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test_set/paper"
resize_images(image_dir_path)

print("보 이미지 resize 완료!")

100  images to be resized.
100  images resized.
가위 이미지 resize 완료!
99  images to be resized.
99  images resized.
바위 이미지 resize 완료!
100  images to be resized.
100  images resized.
보 이미지 resize 완료!


In [20]:
#테스트 이미지 불러오기 #299개로 변환 ! 
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test_set/"
(x_test, y_test)=load_data(image_dir_path, 299)  
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

학습데이터(x_train)의 이미지 개수는 299 입니다.
x_test shape: (299, 28, 28, 3)
y_test shape: (299,)


라벨:  0


In [21]:
import matplotlib.pyplot as plt
plt.imshow(x_test[0])
print('라벨: ', y_test[0])

In [22]:
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 1s - loss: 3.2037 - accuracy: 0.4682
test_loss: 3.203688144683838 
test_accuracy: 0.46822741627693176


In [24]:
#딥러닝 네트워크 설계하기

import tensorflow as tf
from tensorflow import keras
import numpy as np

from tensorflow import keras

#channel : 정해진 값을 가진 건 아님, 
#1과2는 두배 간격이 좋음 (16,32가 정확도가 가장 높게 나옴. )
n_channel_1=16
n_channel_2=32
n_dense=64
n_test_epoch=30

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

print('Model에 추가된 Layer 개수: ', len(model.layers))

model.summary()

Model에 추가된 Layer 개수:  7
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 26, 26, 16)        448       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                51264     
_________________________________________________________________
dense_11 (Dense)             (

In [40]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_test_norm, y_test, epochs=n_test_epoch)


Epoch 1/25
10/10 [==============================] - 0s 3ms/step - loss: 1.0998 - accuracy: 0.3371
Epoch 2/25
10/10 [==============================] - 0s 3ms/step - loss: 1.0857 - accuracy: 0.4515
Epoch 3/25
10/10 [==============================] - 0s 3ms/step - loss: 1.0331 - accuracy: 0.5669
Epoch 4/25
10/10 [==============================] - 0s 4ms/step - loss: 0.9792 - accuracy: 0.6557
Epoch 5/25
10/10 [==============================] - 0s 3ms/step - loss: 0.8760 - accuracy: 0.7811
Epoch 6/25
10/10 [==============================] - 0s 3ms/step - loss: 0.8308 - accuracy: 0.6473
Epoch 7/25
10/10 [==============================] - 0s 3ms/step - loss: 0.7217 - accuracy: 0.7365
Epoch 8/25
10/10 [==============================] - 0s 3ms/step - loss: 0.6469 - accuracy: 0.7986
Epoch 9/25
10/10 [==============================] - 0s 3ms/step - loss: 0.5627 - accuracy: 0.8101
Epoch 10/25
10/10 [==============================] - 0s 3ms/step - loss: 0.4943 - accuracy: 0.8395
Epoch 11/25
10/10 [